# Cardiac-Synchronized Learning Task Implementation
*Author: Hamed Ghane*  
*Date: December 2, 2024*

## Overview
This Python script implements a cognitive learning task synchronized with cardiac cycles using real-time heart signals. The experiment is designed to study how cardiac timing influences learning and decision-making processes, particularly focusing on the differences between systolic and diastolic heart phases.

## Technical Framework
The implementation utilizes several key Python libraries:
- PsychoPy for stimulus presentation and response collection
- Lab Streaming Layer (LSL) for real-time physiological data streaming
- Pandas and NumPy for data management and numerical operations
- Threading for concurrent R-peak detection

## Task Architecture
The experiment is structured as a class-based implementation (`CardiacSyncedLearningTask`) that manages all aspects of the task, from stimulus presentation to data collection. The task incorporates real-time cardiac cycle detection and synchronizes feedback presentation with specific cardiac phases.

### Key Components

1. **Settings Management**
   The task uses configurable parameters including trial counts, timing durations, and probability settings. Notable parameters:
   - Trial duration: 1.25s for decision
   - Feedback duration: 4.0s (calibrated to capture ~4.58 heartbeats)
   - Systole window: 300ms post R-peak

2. **LSL Integration**
   The implementation features two LSL streams:
   - Input: R-peak detection stream for cardiac timing
   - Output: Event markers for experimental synchronization
   All experimental events are marked with descriptive string markers in the data stream.

3. **Trial Structure**
   Each trial follows a precise sequence:
   - Symbol presentation
   - Response collection (1.25s window)
   - Cardiac-synchronized feedback
   - Inter-trial interval (2.0-3.0s)

4. **Experimental Design**
   The task includes three distinct blocks:
   - Systolic feedback timing
   - Diastolic feedback timing
   - Mixed timing (control condition)

### Data Collection
The system captures comprehensive trial-level data including:
- Response times and choices
- Cardiac timing information
- Feedback types and outcomes
- Block and condition information
All data is automatically saved in CSV format with detailed participant information.

## Implementation Notes
The script implements several safeguards and features:
- Continuous R-peak monitoring through threaded collection
- Graceful experiment termination handling
- Comprehensive trial reporting for monitoring
- Flexible timing management for cardiac synchronization

## Usage Requirements
The implementation requires:
- LSL-compatible ECG/R-peak detection system
- PsychoPy environment
- Stimulus images in the specified format
- Appropriate directory structure for data storage

This implementation represents a sophisticated approach to cardiac-behavioral research, enabling precise timing control and comprehensive data collection for studying cardiac-cognitive interactions.

In [ ]:
from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path
from pylsl import StreamInlet, resolve_stream, StreamInfo, StreamOutlet
import threading
import queue
import time

class CardiacSyncedLearningTask:
    def __init__(self):
        # Task settings
        self.settings = {
            'n_trials': 10,  # Trials per block
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 4.0,  # Feedback duration changed to 4.0s
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability': 0.70,  # Probability of win for correct choice
            'systole_duration': 0.300,  # 300ms post R-peak
            'time_resolution': 0.010  # 10ms bins
        }

        # Setup LSL outlet for markers
        self.marker_info = StreamInfo(
            'TaskMarkers',     # Stream name
            'Markers',         # Stream type
            1,                 # Number of channels
            0,                 # Irregular sampling rate
            'string',          # Channel format
            'TaskMarker123'    # Source id
        )
        self.marker_outlet = StreamOutlet(self.marker_info)

        # Marker codes for different events
        self.markers = {
            'experiment_start': 'exp_start',
            'experiment_end': 'exp_end',
            'block_start': 'block_start',
            'block_end': 'block_end',
            'trial_start': 'trial_start',
            'choice_made': 'choice_made',
            'feedback_onset': 'feedback_onset',
            'trial_end': 'trial_end',
            'instruction_start': 'instruct_start',
            'instruction_end': 'instruct_end',
            'timeout': 'timeout',
            'win_feedback': 'feedback_win',
            'loss_feedback': 'feedback_loss',
            'neutral_feedback': 'feedback_neutral'
        }

        # Setup paths
        self.base_path = Path.cwd()
        self.stim_path = self.base_path / 'stimuli'
        self.data_path = self.base_path / 'data'
        self.data_path.mkdir(exist_ok=True)

        # Define stimuli filenames
        self.stimuli = {
            'symbols': ['A.png', 'B.png', 'C.png'],
            'fixation': 'fix.png',
            'feedback': {
                'win': 'win.png',
                'loss': 'loss.png',
                'neutral': 'NEU.png'
            }
        }

        # Task instructions
        self.instructions = [
            """Welcome to the experiment!

In this task, you will see two symbols on each trial.
Your goal is to learn which symbol is more likely to give you rewards.

Press SPACE to continue...""",

            """On each trial:
1. Two symbols will appear
2. Choose the left symbol with the LEFT ARROW key
3. Choose the right symbol with the RIGHT ARROW key
4. You have 1.25 seconds to make your choice

Press SPACE to continue...""",

            """After your choice:
- If you see a WIN image, you earned points
- If you see a LOSS image, you lost points
- If you see a NEUTRAL image, no points change

The same symbols will have different probabilities of winning.
These probabilities may change during the task.

Press SPACE to continue...""",

            """Important Notes:
- Respond as quickly and accurately as possible
- If you don't respond in time, you'll see a neutral outcome
- You can press 'q' at any time to end the experiment
- Your data will be saved automatically

Press SPACE to begin the task."""
        ]

        # LSL and timing variables
        self.r_peak_times = queue.Queue()
        self.current_r_peak = None
        self.next_r_peak = None
        self.cardiac_timing_options = {
            'systole': [],
            'diastole': []
        }

        # Timing mode for the block
        self.timing_modes = ['systole', 'diastole', 'mixed']

    def send_marker(self, marker_code):
        """Send an LSL marker with the specified code"""
        self.marker_outlet.push_sample([marker_code])
        print(f"LSL Marker sent: {marker_code}")

    def setup_experiment(self):
        """Initialize PsychoPy window and load stimuli"""
        self.win = visual.Window(
            size=[1000, 800],
            fullscr=False,
            units='height',
            color=[0, 0, 0],
            allowGUI=True
        )

        # Load stimuli
        self.stim = {
            'symbols': [
                visual.ImageStim(self.win, image=str(self.stim_path / img))
                for img in self.stimuli['symbols']
            ],
            'fixation': visual.ImageStim(
                self.win,
                image=str(self.stim_path / self.stimuli['fixation'])
            ),
            'feedback': {
                key: visual.ImageStim(self.win, image=str(self.stim_path / path))
                for key, path in self.stimuli['feedback'].items()
            },
            'text': visual.TextStim(
                self.win,
                text='',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            ),
            'timeout_msg': visual.TextStim(
                self.win,
                text='Too Slow, Please Answer Quicker',
                height=0.05,
                wrapWidth=0.8,
                color='red'
            )
        }

    def print_trial_report(self, trial_data, trial_num, block_type):
        """Print a detailed report of the trial results"""
        print("\n" + "="*50)
        print(f"Trial {trial_num} Report (Block: {block_type})")
        print("-"*50)

        if trial_data['choice'] is not None:
            choice_str = "Left" if trial_data['choice'] == 0 else "Right"
            print(f"Response: {choice_str} (RT: {trial_data['rt']:.3f}s)")
            print(f"Choice was {'' if trial_data['correct'] else 'in'}correct")
        else:
            print("Response: No response (Too slow)")

        print(f"Feedback: {trial_data['feedback'].upper()}")
        print(f"Cardiac Phase: {trial_data['cardiac_phase']}")
        print(f"Time from R-peak: {trial_data['time_from_r_peak']*1000:.1f}ms")
        print("="*50 + "\n")

    def setup_lsl(self):
        """Setup LSL inlet for R-peak markers"""
        print("Looking for R-peak markers stream...")
        streams = resolve_stream('type', 'R_PEAK')
        self.inlet = StreamInlet(streams[0])

        # Start R-peak collection thread
        self.lsl_thread = threading.Thread(target=self.collect_r_peaks)
        self.lsl_thread.daemon = True
        self.lsl_thread.start()

    def collect_r_peaks(self):
        """Continuously collect R-peak markers"""
        while True:
            sample, timestamp = self.inlet.pull_sample()
            self.r_peak_times.put(timestamp)

    def show_instructions(self):
        """Display task instructions with markers"""
        self.send_marker(self.markers['instruction_start'])

        for instruction in self.instructions:
            self.stim['text'].text = instruction
            self.stim['text'].draw()
            self.win.flip()

            while True:
                keys = event.waitKeys(keyList=['space', 'q'])
                if 'q' in keys:
                    self.win.close()
                    core.quit()
                if 'space' in keys:
                    break

        self.send_marker(self.markers['instruction_end'])

    def get_participant_info(self):
        """Show dialog to collect participant information"""
        current_time = time.strftime("%Y%m%d-%H%M%S")

        exp_info = {
            'participant': '',
            'session': '001',
            'run': '1',
            'date_time': current_time,
        }

        dlg = gui.DlgFromDict(
            dictionary=exp_info,
            title='Task Info',
            fixed=['date_time']
        )

        if dlg.OK:
            return exp_info
        else:
            core.quit()

    def get_cardiac_timing_options(self, r_peak_time, next_r_peak_time):
        """Generate timing options relative to R-peak"""
        options = {'systole': [], 'diastole': []}
        current_time = r_peak_time

        while current_time < next_r_peak_time:
            delay = current_time - r_peak_time
            if delay <= self.settings['systole_duration']:
                options['systole'].append(current_time)
            else:
                options['diastole'].append(current_time)
            current_time += self.settings['time_resolution']

        return options

    def select_feedback_time(self, timing_mode):
        """Select feedback presentation time based on mode"""
        options = []
        if timing_mode == 'systole':
            options = self.cardiac_timing_options['systole']
        elif timing_mode == 'diastole':
            options = self.cardiac_timing_options['diastole']
        else:  # mixed
            options = (self.cardiac_timing_options['systole'] +
                      self.cardiac_timing_options['diastole'])

        return np.random.choice(options)

    def run_trial(self, timing_mode, trial):
        """Run a single trial with cardiac-synced feedback and markers"""
        # Send trial start marker
        self.send_marker(self.markers['trial_start'])

        # Check for quit key
        if 'q' in event.getKeys(keyList=['q']):
            self.win.close()
            core.quit()

        # Decision Phase
        positions = [(-0.15, 0), (0.15, 0)]
        np.random.shuffle(positions)

        for sym, pos in zip(self.stim['symbols'][:2], positions):
            sym.pos = pos
            sym.draw()
        self.win.flip()

        # Get response with timeout
        timer = core.Clock()
        keys = event.waitKeys(
            maxWait=self.settings['decision_duration'],
            keyList=['left', 'right', 'q'],
            timeStamped=timer
        )

        if keys and 'q' in keys[0]:
            self.win.close()
            core.quit()

        # Process response and prepare feedback
        if not keys:  # No response
            self.send_marker(self.markers['timeout'])
            self.stim['timeout_msg'].draw()
            self.win.flip()
            core.wait(1.0)

            feedback_type = 'neutral'
            rt = None
            choice = None
            correct = False
        else:
            self.send_marker(self.markers['choice_made'])
            key, rt = keys[0]
            choice = 0 if key == 'left' else 1

            correct = choice == 0
            if correct:
                outcome = np.random.random() < self.settings['win_probability']
            else:
                outcome = np.random.random() < (1 - self.settings['win_probability'])

            feedback_type = 'win' if outcome else 'loss'

        # Wait for next R-peak and prepare feedback timing
        r_peak = self.r_peak_times.get()
        next_r_peak = self.r_peak_times.get()

        # Generate timing options for this cardiac cycle
        self.cardiac_timing_options = self.get_cardiac_timing_options(
            r_peak, next_r_peak
        )

        # Select feedback time
        feedback_time = self.select_feedback_time(timing_mode)

        # Wait until selected feedback time
        while time.time() < feedback_time:
            core.wait(0.001)

        # Show feedback with appropriate marker
        feedback_marker = {
            'win': self.markers['win_feedback'],
            'loss': self.markers['loss_feedback'],
            'neutral': self.markers['neutral_feedback']
        }[feedback_type]

        self.send_marker(self.markers['feedback_onset'])
        self.send_marker(feedback_marker)

        self.stim['feedback'][feedback_type].draw()
        self.win.flip()
        core.wait(self.settings['feedback_duration'])  # Now 4.0s

        # Send trial end marker
        self.send_marker(self.markers['trial_end'])

        # Create trial data dictionary
        trial_data = {
            'rt': rt,
            'choice': choice,
            'correct': correct,
            'feedback': feedback_type,
            'r_peak_time': r_peak,
            'next_r_peak_time': next_r_peak,
            'feedback_time': feedback_time,
            'cardiac_phase': ('systole' if feedback_time - r_peak <=
                            self.settings['systole_duration'] else 'diastole'),
            'time_from_r_peak': feedback_time - r_peak
        }

        # Print trial report
        self.print_trial_report(trial_data, trial, timing_mode)

        # Inter-trial interval
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(np.random.uniform(
            self.settings['iti_min'],
            self.settings['iti_max']
        ))

        return trial_data

    def run_experiment(self):
        """Run the complete experiment with markers"""
        # Setup
        participant_info = self.get_participant_info()
        self.setup_experiment()

        # Send experiment start marker
        self.send_marker(self.markers['experiment_start'])

        # Setup LSL
        self.setup_lsl()
        print("Calibrating R-peak detection...")
        core.wait(3.0)

        # Show instructions
        self.show_instructions()

        # Create data storage
        all_data = []

        try:
            # Run blocks with different timing modes
            for mode in self.timing_modes:
                # Send block start marker
                self.send_marker(self.markers['block_start'])
                print(f"\nStarting {mode} block...")

                for trial in range(self.settings['n_trials']):
                    trial_data = self.run_trial(mode, trial)
                    trial_data.update({
                        'trial': trial,
                        'timing_mode': mode,
                        'participant': participant_info['participant'],
                        'session': participant_info['session'],
                        'run': participant_info['run'],
                        'date_time': participant_info['date_time']
                    })
                    all_data.append(trial_data)

                # Send block end marker
                self.send_marker(self.markers['block_end'])

        finally:
            # Send experiment end marker
            self.send_marker(self.markers['experiment_end'])

            if all_data:
                df = pd.DataFrame(all_data)
                filename = f"{participant_info['participant']}-ses{participant_info['session']}-run{participant_info['run']}-{participant_info['date_time']}.csv"
                df.to_csv(self.data_path / filename, index=False)
                print(f"\nData saved as: {filename}")

            self.win.close()

if __name__ == "__main__":
    task = CardiacSyncedLearningTask()
    task.run_experiment()